# Laboratorio 3 - Parte Calificada
## Lewis Rengifo - 20160877

### Pregunta 1
Considerar la media aritmetica
La frecuencia de la CPU que se nos muestra luego de ejecutar el comando lscpu en linux es 2.90 GHz

In [1]:
! lscpu

Arquitectura:                            x86_64
  modo(s) de operación de las CPUs:      32-bit, 64-bit
  Address sizes:                         48 bits physical, 48 bits virtual
  Orden de los bytes:                    Little Endian
CPU(s):                                  16
  Lista de la(s) CPU(s) en línea:        0-15
ID de fabricante:                        AuthenticAMD
  Nombre del modelo:                     AMD Ryzen 7 2700X Eight-Core Processor
    Familia de CPU:                      23
    Modelo:                              8
    Hilo(s) de procesamiento por núcleo: 2
    Núcleo(s) por «socket»:              8
    «Socket(s)»                          1
    Revisión:                            2
    BogoMIPS:                            7386.13
    Indicadores:                         fpu vme de pse tsc msr pae mce cx8 apic
                                          sep mtrr pge mca cmov pat pse36 clflus
                                         h mmx fxsr sse sse2 ht syscall 

### Pregunta 2

In [2]:
%%file suma_cuadrados.c
#include <stdio.h>
#include <math.h>
#include <stdlib.h>

double suma_cuadrados(double* l, int N){
    double suma = 0.0;
    for (int i = 1; i <= N; i++){
        double cuadrado = pow(l[i],2);
        suma += cuadrado;
    }
    return suma;
    
}

int main(){
    return 0;
}

Overwriting suma_cuadrados.c


### Pregunta 3

In [3]:
! gcc suma_cuadrados.c -o suma_cuadrados.o -lm -O

### Pregunta 4

In [4]:
! objdump -d suma_cuadrados.o


suma_cuadrados.o:     formato del fichero elf64-x86-64


Desensamblado de la sección .init:

0000000000001000 <_init>:
    1000:	f3 0f 1e fa          	endbr64 
    1004:	48 83 ec 08          	sub    $0x8,%rsp
    1008:	48 8b 05 d9 2f 00 00 	mov    0x2fd9(%rip),%rax        # 3fe8 <__gmon_start__@Base>
    100f:	48 85 c0             	test   %rax,%rax
    1012:	74 02                	je     1016 <_init+0x16>
    1014:	ff d0                	call   *%rax
    1016:	48 83 c4 08          	add    $0x8,%rsp
    101a:	c3                   	ret    

Desensamblado de la sección .plt:

0000000000001020 <.plt>:
    1020:	ff 35 a2 2f 00 00    	push   0x2fa2(%rip)        # 3fc8 <_GLOBAL_OFFSET_TABLE_+0x8>
    1026:	f2 ff 25 a3 2f 00 00 	bnd jmp *0x2fa3(%rip)        # 3fd0 <_GLOBAL_OFFSET_TABLE_+0x10>
    102d:	0f 1f 00             	nopl   (%rax)

Desensamblado de la sección .plt.got:

0000000000001030 <__cxa_finalize@plt>:
    1030:	f3 0f 1e fa          	endbr64 
    1034:	f2 ff 25 bd 2f 00 00 	bnd jmp

### Pregunta 5

In [5]:
%%file main.c
#include <stdio.h>
#include <math.h>
#include <stdlib.h>
#include <time.h>
#include <x86intrin.h>
#include <stdint.h>

int* crear_arr(int N){
    int* arreglo = (int*)malloc(N * sizeof (int));
    srand (time(NULL));
    for  (int i=0; i <= N; i++){
        arreglo[i] = rand()%10;
    }
    return arreglo;
}

double suma_cuadrados(int* l, int N){
    double suma = 0.0;
    for (int i = 1; i <= N; i++){
        double cuadrado = pow(l[i],2);
        suma += cuadrado;
    }
    return suma;
    
}

int num_instrucciones (int N){
    int cant_instrucciones;
    cant_instrucciones = 7 + (7*N);
    return cant_instrucciones;
}

int ret_cpi (int ciclos, int cant_instrucciones){
    int cpi = ciclos * cant_instrucciones;
    return cpi;
}

double ret_mips(double frec_reloj, int cpi){
    double mips = frec_reloj/((double)cpi * 10e6);
    return mips;
}

void test (int N){
    double cpis[100000-1];
    double mips_s[100000-1];
    //los ciclos se calculan cuanto se demora en ejecutar la funcion suma_cuadrados
    int a, b;
    int* l = crear_arr(N);
    for (int i = 0; i<N;i++){
        printf("mi arreglo es %d\n", l[i]);
    }
    a = __rdtsc();
    suma_cuadrados(l, N);
    b = __rdtsc();
    int ciclos = (int)b - (int)a;
    printf("cantidad de ciclos: %d\n", ciclos);
    // instrucciones
    int instrucciones = num_instrucciones (N);
    printf("cantidad de instrucciones: %d\n", instrucciones);
    //frecuencia reloj
    int frec_reloj = (48000000 - 8000000)/2;
    printf("frec del reloj: %d\n", frec_reloj);
    //iteracion para cpi
    for (int i = 0; i < N; i++){
        //crear un arreglo que guarde los resultados de los calculos
        cpis[i] = ret_cpi(ciclos, instrucciones);
    }
    printf("primer valor de cpi: %f\n", cpis[0]);
    printf("segundo valor del cpi: %f\n", cpis[1]);
    //iteracion para mips
    for (int i = 0; i < N; i++){
        //crear un arreglo que guarde los resultados de los calculos
        mips_s[i]=ret_mips(frec_reloj, cpis[i]);
    }
    //imprimir
    int total_cpi = 0;
    int total_mips = 0;
    int prom_cpi, prom_mips;
    int mayor_cpi, menor_cpi;
    int mayor_mips, menor_mips;
    for (int i = 0; i < N; i++){
        total_cpi += cpis[i];
        total_mips += mips_s[i];
        if (cpis[i] > mayor_cpi) mayor_cpi = cpis[i];
		if (cpis[i] < menor_cpi) menor_cpi = cpis[i];
        if (mips_s[i] > mayor_mips) mayor_mips = mips_s[i];
		if (mips_s[i] < menor_mips) menor_mips = mips_s[i];
    }
    prom_cpi = abs(total_cpi/10000);
    prom_mips = abs(total_mips/100000);
    printf("El mayor cpi es %d y el menor cpi es %d\n", mayor_cpi, menor_cpi);
    printf("El promedio de cpi es %d\n", prom_cpi);
    printf("El promedio de mips es %d\n", prom_mips);
    printf("El mayor mips es %d y el menor mibs es %d\n", mayor_mips, menor_mips);

}
int main(int argc, char *argv[]){
    if (argc<=1){
        printf("El uso del programa es correcto\n");
        int N = 16384;
        test (N);
        return 0;
    } else if (argc>2){
        printf ("No es valido ingresar dos argumentos o mas, por favor ingresar un solo argumento o ninguno\n");
        return 0;
    } else {
        printf("El uso del programa es correcto\n");
        int N = atoi(argv[1]);
        test(N);
        return 0;
    }
    
}

Overwriting main.c


### Pregunta 6

In [6]:
! gcc main.c -o main -lm

### Pregunta 7

In [7]:
! ./main

El uso del programa es correcto
mi arreglo es 7
mi arreglo es 7
mi arreglo es 9
mi arreglo es 5
mi arreglo es 1
mi arreglo es 8
mi arreglo es 5
mi arreglo es 8
mi arreglo es 9
mi arreglo es 7
mi arreglo es 9
mi arreglo es 3
mi arreglo es 3
mi arreglo es 1
mi arreglo es 3
mi arreglo es 2
mi arreglo es 6
mi arreglo es 0
mi arreglo es 7
mi arreglo es 3
mi arreglo es 3
mi arreglo es 6
mi arreglo es 8
mi arreglo es 9
mi arreglo es 3
mi arreglo es 8
mi arreglo es 3
mi arreglo es 8
mi arreglo es 7
mi arreglo es 3
mi arreglo es 8
mi arreglo es 6
mi arreglo es 3
mi arreglo es 7
mi arreglo es 1
mi arreglo es 6
mi arreglo es 8
mi arreglo es 9
mi arreglo es 4
mi arreglo es 7
mi arreglo es 6
mi arreglo es 6
mi arreglo es 2
mi arreglo es 1
mi arreglo es 7
mi arreglo es 5
mi arreglo es 4
mi arreglo es 3
mi arreglo es 7
mi arreglo es 1
mi arreglo es 7
mi arreglo es 0
mi arreglo es 9
mi arreglo es 7
mi arreglo es 0
mi arreglo es 2
mi arreglo es 7
mi arreglo es 5
mi arreglo es 3
mi arreglo es 6
mi arreg

In [8]:
! pwd

/home/lab3
